# Feature 2: Data Cleaning

In [1]:
import pandas as pd
import re
import csv
from fractions import Fraction
import builtins

## Regular Expression

In [2]:
def parseNumber(s):
    return float(Fraction(re.sub(r'\s+', '', s)))

def cleanWebScraping(rawFileName,cleanFileName):
    raw_csv = pd.read_csv(rawFileName, index_col=0)
    inter_dict = raw_csv.to_dict("split")
    raw_dict = builtins.dict(zip(inter_dict["index"], inter_dict["data"]))
    
    ls = ([["drink_name","ingredient","measurement","unit"]])
    for key in raw_dict:
        vl = raw_dict.get(key)
        whitelist = set('abcdefghijklmnopqrstuvwxy ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        key = ''.join(filter(whitelist.__contains__, key))
        for v in vl:
            if pd.isnull(v) == False:
            #regular expression to deal with number/fractions + list of usual measurements
                n = re.search(r'(\d(?:[\d\./\s/\.]*\d)?)\s*(.*(?:(?:cup|tablespoon|can|ounce|teaspoon|pint|jigger|shot|liter|bottle|oz|oz.\s)s?|(?:dash|glass|inch)(?:es)?)\b|)\s*(.*)', v)
                if pd.isnull(n):
                    ls.append([key,v,'N/A','N/A'])
                else:
                    quantity = parseNumber(n.group(1))
                    units = n.group(2)
                    u = re.search(r'\(\s*(\d(?:[\d./\s]*\d)?)\s*([^)]*)\)', units)
                    if not pd.isnull(u):
                        quantity *= parseNumber(u.group(1))
                        units = u.group(2)
                    ls.append([key,n.group(3),quantity,units])
                
            
    #output the cleaned csv
    with open(cleanFileName, "w") as f:
        writer = csv.writer(f)
        writer.writerows(ls)
        
        
def cleanAPI(rawFileName,cleanFileName):
    raw_csv = pd.read_csv(rawFileName, index_col=0)
    inter_dict = raw_csv.to_dict("split")
    raw_dict = builtins.dict(zip(inter_dict["index"], inter_dict["data"]))
    
    ls = ([["drink_name","ingredient","measurement","unit"]])
    for key in raw_dict:
        vl = raw_dict.get(key)
        #whitelist = set('abcdefghijklmnopqrstuvwxy ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        #key = ''.join(filter(whitelist.__contains__, key))
        #for v in vl:
        for i in range(0,len(vl)-1,2):
            if pd.isnull(vl[i]) == False:
            #regular expression to deal with number/fractions + list of usual measurements
                n = re.search(r'(\d(?:[\d\./\s/\.]*\d)?)\s*(.*(?:(?:cup|tablespoon|can|ounce|teaspoon|pint|jigger|shot|liter|bottle|oz|oz.\s)s?|(?:dash|glass|inch)(?:es)?)\b|)\s*(.*)', str(vl[(i+1)]))
                if pd.isnull(n):
                    if type(vl[i+1]) is float and vl[i+1] is not None: 
                        ls.append([key,vl[i],vl[i+1],'N/A'])
                    else:
                        ls.append([key,vl[i],'N/A','N/A'])
                else:
                    quantity = parseNumber(n.group(1))
                    units = n.group(2)
                    u = re.search(r'\(\s*(\d(?:[\d./\s]*\d)?)\s*([^)]*)\)', units)
                    if not pd.isnull(u):
                        quantity *= parseNumber(u.group(1))
                        units = u.group(2)
                    ls.append([key,vl[i],quantity,units])
                
            
    #output the cleaned csv
    with open(cleanFileName, "w") as f:
        writer = csv.writer(f)
        writer.writerows(ls)

In [3]:
cleanWebScraping("allrecipes_raw.csv","allrecipes_clean.csv")
cleanWebScraping("epicurious_raw.csv","epicurious_clean.csv")
cleanAPI("cocktaildb_raw.csv","cocktaildb_clean.csv")

## Merge Files

In [4]:
#add the cover sheet
ls = ([["sheet_name","data_source","Description"]])
ls.append(['allrecipes_raw','allrecipes.com','Raw data from scraping the allrecipes.com website with categorized base liquors'])
ls.append(['allrecipes_clean','','cleaned data from the sheet described above with drink name and each of the ingredient with measurement in every row'])
ls.append(['epicurious_raw','epicurious.com','Raw data from scraping the epicurious.com website with categorized base liquors'])
ls.append(['epicurious_clean','','cleaned data from the sheet described above with drink name and each of the ingredient with measurement in every row'])
ls.append(['cocktaildb_raw','https://www.thecocktaildb.com/api/json/v1/1/random.php','a web API that generate one random drink at a time with 1'])
ls.append(['cocktaildb_clean','','cleaned data from the sheet described above with drink name and each of the ingredient with measurement in every row'])
ls.append(['merged','','merged data sheet combining allrecipes_clean, epicurious_clean and cocktaildb_clean'])
ls.append(['merged_2','','merged data sheet combining allrecipes_raw, epicurious_raw  cocktaildb_raw'])
with open("cover.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(ls)

In [5]:
#add the merged sheet
df1 = pd.read_csv('allrecipes_clean.csv')
df2 = pd.read_csv('epicurious_clean.csv')
df3 = pd.read_csv('cocktaildb_clean.csv')

out = df1.append(df2)
out = out.append(df3)
out.to_csv('merged.csv', sep=',', encoding='utf-8')

In [9]:
df4 = pd.read_csv('allrecipes_raw.csv')
df5 = pd.read_csv('epicurious_raw.csv')
df6 = pd.read_csv('cocktaildb_raw.csv')
out2 = df4.append(df5)
out2 = out2.append(df6)
out2.to_csv('merged_2.csv', sep=',', encoding='utf-8')

In [10]:
#write every sheet into the same Workbook
writer = pd.ExcelWriter('Workbook.xlsx', engine='xlsxwriter')

#read all csv files
cover_df = pd.read_csv('cover.csv')
allrecipes_raw_df = pd.read_csv('allrecipes_raw.csv')
#allrecipes_clean_df = pd.read_csv('allrecipes_clean.csv')
cocktaildb_raw_df = pd.read_csv('cocktaildb_raw.csv')
#cocktaildb_clean_df = pd.read_csv('cocktaildb_clean.csv')
epicurious_raw_df = pd.read_csv('epicurious_raw.csv')

#merged_df = pd.read_csv('merged.csv')
#write all csv files
cover_df.to_excel(writer, sheet_name='cover')
allrecipes_raw_df.to_excel(writer, sheet_name='allrecipes_raw')
df1.to_excel(writer, sheet_name='allrecipes_clean')
epicurious_raw_df.to_excel(writer, sheet_name='epicurious_raw')
df2.to_excel(writer,sheet_name = 'epicurious_clean')
cocktaildb_raw_df.to_excel(writer, sheet_name='cocktaildb_raw')
df3.to_excel(writer, sheet_name='cocktaildb_clean')
out.to_excel(writer,sheet_name='merged')
out2.to_excel(writer,sheet_name='merged_2')
writer.save()